In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
class SimpleNeuralNetworkModel:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.rand(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def ReLu(self, x):
        return np.maximum(0, x)
    
    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def forward_propagation(self, x):
        self.z1 = np.dot(x, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        

In [16]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

data = [
    [1,0,1,0,1],
    [1,0,1,0,1],
    [1,0,1,0,1],
    [1,0,1,0,1],
    [1,0,1,0,1],
]

W1 = np.random.randn(5, 5)
b1 = np.zeros((1, 5))
z1 = np.dot(data, W1) + b1
a1 = sigmoid(z1)
print(W1,'\n')
print(b1,'\n')
print(z1,'\n')
print(a1,'\n')

[[-0.43175526  0.01905682 -0.82347681 -0.03037417 -1.36111442]
 [-0.51967033 -0.42379929  1.37603507 -0.74186663 -0.67332874]
 [ 0.25741542 -0.97814406 -1.16012642  1.50009201  0.39034376]
 [-0.93342257  2.43310176  1.69620139 -0.62705832  0.97770959]
 [ 0.93861008 -0.28319048  1.05843852 -1.31483947 -0.43248124]] 

[[0. 0. 0. 0. 0.]] 

[[ 0.76427024 -1.24227772 -0.92516471  0.15487836 -1.4032519 ]
 [ 0.76427024 -1.24227772 -0.92516471  0.15487836 -1.4032519 ]
 [ 0.76427024 -1.24227772 -0.92516471  0.15487836 -1.4032519 ]
 [ 0.76427024 -1.24227772 -0.92516471  0.15487836 -1.4032519 ]
 [ 0.76427024 -1.24227772 -0.92516471  0.15487836 -1.4032519 ]] 

[[0.68228013 0.22403976 0.28390672 0.53864238 0.19730059]
 [0.68228013 0.22403976 0.28390672 0.53864238 0.19730059]
 [0.68228013 0.22403976 0.28390672 0.53864238 0.19730059]
 [0.68228013 0.22403976 0.28390672 0.53864238 0.19730059]
 [0.68228013 0.22403976 0.28390672 0.53864238 0.19730059]] 

